In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [2]:
%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [3]:
from IPython.display import HTML

HTML('''<script>
code_show=true; 
function code_toggle() {
 if (code_show){
 $('div.input').hide();
 } else {
 $('div.input').show();
 }
 code_show = !code_show
} 
$( document ).ready(code_toggle);
</script>
<form action="javascript:code_toggle()"><input type="submit" value="Click here to toggle on/off the raw code."></form>''')

# Evaluation of model DIC and TA bias matched to salinity

**Notebook overview:** We understand that our carbon model, SalishSeaCast-SKØG, has a slight bias ($\sim$+25 umol/L) in the total alkalinity field. This small bias results in a non-negligible bias in the environmentally-relevant $\Omega_A$ metric that we often consider ($\sim$+0.2). However

In [4]:
#imports
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
import seawater
import netCDF4 as nc
from scipy import stats

from numpy import *
from scipy import *

import netCDF4 as nc
import numpy as np
import scipy as sp
import cmocean
import glob
import seawater
import arrow
from salishsea_tools import (
    nc_tools,
    viz_tools,
    geo_tools,
    tidetools
)

import warnings
warnings.filterwarnings('ignore')

In [5]:
infil = np.loadtxt('./Datasets/grl2016_nu.txt')

crid= infil[:,0]
ln = infil[:,2]
stn = infil[:,3]
mon = infil[:,4]
day = infil[:,5]
lat_or = infil[:,6]
lon_or = infil[:,7]
lat = infil[:,6]
lon = infil[:,7]
P = infil[:,8]
T = infil[:,9]
S = infil[:,10]
ox = infil[:,11]
ox_qf = infil[:,12]
dic = infil[:,13]
alk = infil[:,15]
dic_qf = infil[:,14]
alk_qf = infil[:,16]
no3 = infil[:,17]
no3_qf = infil[:,18]
si = infil[:,19]
si_qf = infil[:, 20]
po4 = infil[:,21]
po4_qf = infil[:, 22]



##add model coords
obs_i = np.zeros_like(alk)
obs_j = np.zeros_like(alk)
moddepth = np.zeros_like(alk)
depthdist_modobs = np.zeros_like(alk)

mesh = nc.Dataset('/data/tjarniko/MEOPAR/grid/bathymetry_201702.nc')
grid = nc.Dataset('/data/tjarniko/MEOPAR/grid/mesh_mask201702.nc')
nav_lon = mesh.variables['nav_lon'][:]
nav_lat = mesh.variables['nav_lat'][:]
bathy = mesh.variables['Bathymetry'][:]
depth = grid.variables['gdept_1d'][:]

dist = []
for w in range(0,len(lat)):
    j, i = geo_tools.find_closest_model_point(lon[w],lat[w],nav_lon,nav_lat)
    obs_j[w] = j
    obs_i[w] = i
    close = np.abs(depth-P[w])
    # smallest
    depthdist_modobs[w] = (np.min(close))
    moddepth[w] = (np.argmin(close))
    

In [6]:
mask = (((dic_qf==2) | (dic_qf==6)) & \
        ((alk_qf==2) | (alk_qf==6)) & \
        (dic > 0) & (alk >0))
print(np.shape(dic[mask]))

### get accessable subset


dens = seawater.dens(S,T,P)
sal_conv = S * 35.16504/35
# CONVERT FROM UMOL TO MMOL/M3
ta_conv=alk*dens/1000
dic_conv=dic*dens/1000
sal_conv = S * 35.16504/35

### subset, converted to model units
dic_SSconv = dic_conv[mask]
ta_SSconv = ta_conv[mask]
sal_SSconv = sal_conv[mask]
depth_SSconv = P[mask]; lat_SSconv = lat[mask]; lon_SSconv = lon[mask]
mon_SSconv = mon[mask]; day_SSconv = day[mask]; i_SSconv = obs_i[mask].astype(int);
j_SSconv = (obs_j[mask].astype(int))
moddepth_SSconv = moddepth[mask]; depthdist_modobs_SSconv = depthdist_modobs[mask]


(425,)


In [7]:
def find_model_dat_bysal(w, yr):
    
    sal_marker = 1
    lb_loc = np.nan; ub_loc = np.nan
    TA_int = np.nan; DIC_int = np.nan; temp_int = np.nan
    
    #find data file 
    mon = mon_SSconv[w]; day = day_SSconv[w]
    tdate = f'{yr}-{int(mon)}-{int(day)}'
    td = arrow.get(tdate)
    ymd = td.format('YYYYMMDD')
    #print(ymd)
    
    #open the relevant carp and grid files
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/carp_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    carpnc = nc.Dataset(ptd2[0])
    ptd = f'/data/tjarniko/results/hindcast.201905_dayavg_DIC-TA-T-S/grid_1d_{ymd}.nc'
    ptd2 = glob.glob(ptd)
    gridnc = nc.Dataset(ptd2[0])
    
    # extract salinity, temperature, etc
    salvec = gridnc['model_output']['SAL'][:,j_SSconv[w],i_SSconv[w]]
    salvec[salvec == 0] = np.nan
    
    tempvec = gridnc['model_output']['TEMP'][:,j_SSconv[w],i_SSconv[w]]
    tempvec[tempvec == 0] = np.nan

    TAvec = carpnc['model_output']['TA'][:,j_SSconv[w],i_SSconv[w]]
    TAvec[TAvec == 0] = np.nan

    DICvec = carpnc['model_output']['DIC'][:,j_SSconv[w],i_SSconv[w]]
    DICvec[DICvec == 0] = np.nan
    

    try:
        sal_lb = (np.max(salvec[salvec<sal_SSconv[w]]))
        lb_loc = np.where(salvec == sal_lb)
        lb_loc = lb_loc[0][0]
    except:
        #print(f'obs fresher than all mod points here, mod min sal is:{np.nanmin(salvec)}')
        sal_marker = 2
    ### min of things higher than the obs
    try:
        sal_ub = (np.min(salvec[salvec>sal_SSconv[w]]))
        ub_loc = np.where(salvec == sal_ub)
        ub_loc = ub_loc[0][0]
    except:
        #print(f'obs saltier than all mod points here, mod max sal is:{np.nanmax(salvec)}')
        sal_marker = 3

    if sal_marker == 1:
        
        #finding lower (lb) and upper (ub) bounds
        TA_lb = TAvec[lb_loc]; DIC_lb = DICvec[lb_loc]; temp_lb = tempvec[lb_loc]; 
        TA_ub = TAvec[ub_loc]; DIC_ub = DICvec[ub_loc]; temp_ub = tempvec[ub_loc]; 
        
        
        saldiff = sal_ub-sal_lb
        #where between the two salinity model measurements are we (interpolation)
        sal_extraprop = (sal_SSconv[w] - sal_lb)/saldiff
        
        #interpolation of TA, etc
        TAdiff = TA_ub-TA_lb; 
        TA_int = TA_lb + sal_extraprop * TAdiff
        
        DICdiff = DIC_ub-DIC_lb; 
        DIC_int = DIC_lb + sal_extraprop * DICdiff
        
        tempdiff = temp_ub-temp_lb; 
        temp_int = temp_lb + sal_extraprop * tempdiff
        
    gridnc.close(); carpnc.close()
    
    return lb_loc, ub_loc, sal_marker, TA_int, DIC_int, temp_int
    



In [ ]:
# lb_loc_ar = np.zeros(425)
# ub_loc_ar = np.zeros(425)
# sal_marker_ar = np.zeros(425)
# TA_int_ar = np.zeros(425)
# DIC_int_ar = np.zeros(425)
# temp_int_ar = np.zeros(425)

# for i in range(0,425):
#     if i%10 == 0: 
#         print(i)
#     lb_loc_ar[i], ub_loc_ar[i], sal_marker_ar[i],\
#     TA_int_ar[i], DIC_int_ar[i], temp_int_ar[i] = find_model_dat_bysal(i,2015)

In [ ]:
# lb_loc_ar = np.zeros(425)
# ub_loc_ar = np.zeros(425)
# sal_marker_ar = np.zeros(425)
# TA_int_ar = np.zeros(425)
# DIC_int_ar = np.zeros(425)
# temp_int_ar = np.zeros(425)

In [ ]:
# ncname = 'YR2015_salspace_modeval.nc'
# f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
# g = f.createGroup('model_output')
# g.createDimension('ref', 425)
# ts = g.createVariable('model_j','f4',('ref'))
# ts[:] = j_SSconv[:]
# ts = g.createVariable('model_i','f4',('ref'))
# ts[:] = i_SSconv[:]
# ts = g.createVariable('lb_loc','f4',('ref'))
# ts[:] = lb_loc_ar[:]
# ts2 = g.createVariable('ub_loc','f4',('ref'))
# ts2[:] = ub_loc_ar[:]
# ts3 = g.createVariable('sal_marker','f4',('ref'))
# ts3[:] = sal_marker_ar[:]
# ts3 = g.createVariable('TA_int','f4',('ref'))
# ts3[:] = TA_int_ar[:]
# ts3 = g.createVariable('DIC_int','f4',('ref'))
# ts3[:] = DIC_int_ar[:]
# ts3 = g.createVariable('temp_int','f4',('ref'))
# ts3[:] = temp_int_ar[:]
# f.close()

In [ ]:
np.nanmean(TA_ints[~np.isnan(TA_ints)]) - np.nanmean(ta_SSconv[~np.isnan(TA_ints)])

In [ ]:
np.size(sal_marker_ar[sal_marker_ar == 1])

In [ ]:
# ### subset, converted to model units
# dic_SSconv = dic_conv[mask]
# ta_SSconv = ta_conv[mask]
# sal_SSconv = sal_conv[mask]
# depth_SSconv = P[mask]; lat_SSconv = lat[mask]; lon_SSconv = lon[mask]
# mon_SSconv = mon[mask]; day_SSconv = day[mask]; i_SSconv = obs_i[mask].astype(int);
# j_SSconv = (obs_j[mask].astype(int))
# moddepth_SSconv = moddepth[mask]; depthdist_modobs_SSconv = depthdist_modobs[mask]




In [17]:

# f = nc.Dataset(ncname,'w', format='NETCDF4') #'w' stands for write
# g = f.createGroup('model_output')
# g.createDimension('ref', 425)
# ts = g.createVariable('model_j','f4',('ref'))
# ts[:] = j_SSconv[:]
# ts = g.createVariable('model_i','f4',('ref'))
# ts[:] = i_SSconv[:]
# ts = g.createVariable('lb_loc','f4',('ref'))
# ts[:] = lb_loc_ar[:]
# ts2 = g.createVariable('ub_loc','f4',('ref'))
# ts2[:] = ub_loc_ar[:]
# ts3 = g.createVariable('sal_marker','f4',('ref'))
# ts3[:] = sal_marker_ar[:]
# ts3 = g.createVariable('TA_int','f4',('ref'))
# ts3[:] = TA_int_ar[:]
# ts3 = g.createVariable('DIC_int','f4',('ref'))
# ts3[:] = DIC_int_ar[:]
# ts3 = g.createVariable('temp_int','f4',('ref'))
# ts3[:] = temp_int_ar[:]
# f.close()

ncname = 'YR2015_salspace_modeval.nc'
w = nc.Dataset(ncname)

yr15_DIC = w['model_output']['DIC_int'][:]
yr15_TA = w['model_output']['TA_int'][:]

ncname = 'YR2014_salspace_modeval.nc'
w = nc.Dataset(ncname)

yr14_DIC = w['model_output']['DIC_int'][:]
yr14_TA = w['model_output']['TA_int'][:]

ncname = 'YR2013_salspace_modeval.nc'
w = nc.Dataset(ncname)

yr13_DIC = w['model_output']['DIC_int'][:]
yr13_TA = w['model_output']['TA_int'][:]

In [22]:

print(np.shape(yr13_DIC[np.isnan(yr13_DIC)]))
print(np.shape(yr14_DIC[np.isnan(yr14_DIC)]))
print(np.shape(yr15_DIC[np.isnan(yr15_DIC)]))

(83,)
(85,)
(94,)


In [27]:
dic_SSconv13 = np.copy(dic_SSconv)
dic_SSconv13[np.isnan(yr13_DIC)] = np.nan
dic_SSconv14 = np.copy(dic_SSconv)
dic_SSconv14[np.isnan(yr14_DIC)] = np.nan
dic_SSconv15 = np.copy(dic_SSconv)
dic_SSconv15[np.isnan(yr15_DIC)] = np.nan

#model minus obs
print('bias (salinity matched) 2013-dic')
bias_dic13 = np.nanmean(yr13_DIC) - (np.nanmean(dic_SSconv13)) 
print(bias_dic13)
print('bias (salinity matched) 2014-dic')
bias_dic14 = np.nanmean(yr14_DIC) - (np.nanmean(dic_SSconv14)) 
print(bias_dic14)
print('bias (salinity matched) 2015-dic')
bias_dic15 = np.nanmean(yr15_DIC) - (np.nanmean(dic_SSconv15)) 
print(bias_dic15)

ta_SSconv13 = np.copy(ta_SSconv)
ta_SSconv13[np.isnan(yr13_TA)] = np.nan
ta_SSconv14 = np.copy(ta_SSconv)
ta_SSconv14[np.isnan(yr14_TA)] = np.nan
ta_SSconv15 = np.copy(ta_SSconv)
ta_SSconv15[np.isnan(yr15_TA)] = np.nan

#model minus obs
print('bias (salinity matched) 2013-ta')
bias_ta13 = np.nanmean(yr13_TA) - (np.nanmean(ta_SSconv13)) 
print(bias_ta13)
print('bias (salinity matched) 2014-ta')
bias_ta14 = np.nanmean(yr14_TA) - (np.nanmean(ta_SSconv14)) 
print(bias_ta14)
print('bias (salinity matched) 2015-ta')
bias_ta15 = np.nanmean(yr15_TA) - (np.nanmean(ta_SSconv15)) 
print(bias_ta15)

bias (salinity matched) 2013-dic
-16.84225871442004
bias (salinity matched) 2014-dic
-12.378087612676154
bias (salinity matched) 2015-dic
-13.099251436387476
bias (salinity matched) 2013-ta
9.714985613101362
bias (salinity matched) 2014-ta
12.437008558263642
bias (salinity matched) 2015-ta
12.491253158983454
